In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.io import arff
import data_processing as dp

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
data = arff.loadarff("../data/5year.arff")
df = pd.DataFrame(data[0])
X_train, X_test, y_train, y_test = dp.get_train_test(df)

In [29]:
X_train.info

<bound method DataFrame.info of          Attr5     Attr6     Attr9    Attr13    Attr15    Attr20    Attr21  \
4367  0.462656  0.461058  0.065347  0.117094  0.878083  0.004728  0.017466   
4356  0.462687  0.460960  0.065011  0.117104  0.877796  0.004388  0.017456   
5552  0.462641  0.460601  0.089876  0.117065  0.877588  0.007469  0.017486   
5906  0.462617  0.460424  0.067593  0.117058  0.877439  0.006667  0.017441   
408   0.462699  0.460601  0.066595  0.117224  0.877775  0.008136  0.017452   
...        ...       ...       ...       ...       ...       ...       ...   
425   0.462730  0.460601  0.071241  0.117136  0.877763  0.006279  0.017471   
4457  0.462693  0.460843  0.065271  0.117099  0.877797  0.003784  0.017458   
3500  0.462688  0.460637  0.065745  0.117113  0.878003  0.006548  0.017487   
779   0.462615  0.460282  0.064361  0.117082  0.876064  0.009333  0.017489   
1515  0.462647  0.460601  0.082247  0.117111  0.877906  0.009229  0.017489   

        Attr27    Attr29    Att

In [30]:
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train, y_train)

c:\Users\65835\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=0, solver='liblinear')

In [31]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[1106,    0],
       [   0,   76]], dtype=int64)

In [32]:
model.score(X_train, y_train)

1.0

In [33]:
model.score(X_test, y_test)

1.0